In [31]:
import os

In [32]:
%pwd

'D:\\Projects\\ChickenDisease\\Chicken-Disease'

In [33]:
os.chdir("D:\Projects\ChickenDisease\Chicken-Disease")

In [34]:
%pwd

'D:\\Projects\\ChickenDisease\\Chicken-Disease'

In [35]:
from dataclasses import dataclass
from pathlib import Path
# dataclasses are immutable, so they can't be changed
@dataclass(frozen=True) # frozen=True means that the dataclass can't be changed
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [36]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [37]:
class ConfigurationManager:
    def __init__(
            self,
            config_file_path = CONFIG_FILE_PATH,
            params_file_path = PARAMS_FILE_PATH):
            

            self.config = read_yaml(config_file_path)
            self.params = read_yaml(params_file_path)

            create_directories([self.config.artifacts_root]) # artifacts_root returns value from configbox in read_yaml
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
          config = self.config.data_ingestion

          create_directories([config.root_dir])

          data_ingestion_config = DataIngestionConfig(
                root_dir= config.root_dir,
                source_url= config.source_url,
                local_data_file=config.local_data_file,
                unzip_dir=config.unzip_dir
          )

          return data_ingestion_config


In [38]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [42]:
class DataIngestion:
    def __init__(self, config=DataIngestionConfig): 
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_url,
                filename= self.config.local_data_file
            )
            logger.info(f"Downloaded {filename} to {self.config.local_data_file} with following info:\n{headers}")
        else:
            logger.info(f"File {self.config.local_data_file} already exists of size : {get_size(Path(self.config.local_data_file))}")
    
    def extract_zip_file(self):
        """
        Extracts the zip file to the data directory
        Function returns none
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [43]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-08-29 19:15:25,472:INFO: common:yaml file : config\config.yaml loaded successfully]
[2023-08-29 19:15:25,476:INFO: common:yaml file : params.yaml loaded successfully]
[2023-08-29 19:15:25,479:INFO: common:Directory at : artifacts created successfully]
[2023-08-29 19:15:25,481:INFO: common:Directory at : artifacts/data_ingestion created successfully]
[2023-08-29 19:15:25,487:INFO: 1407035461:File artifacts/data_ingestion/ChickenDiseaseDataset.zip already exists of size : 17677 kb]
